In [22]:
import openeo
import numpy as np

url = "https://openeo.dataspace.copernicus.eu"
connection = openeo.connect(url).authenticate_oidc()

Authenticated using refresh token.


In [23]:
datacube = openeo.rest.datacube.DataCube(
  openeo.rest.datacube.PGNode(
    "insar_preprocessing",
    arguments={
      "burst_id": 249435,
      "sub_swath": "IW2",
      "InSAR_pairs": [
        ["2024-08-09", "2024-08-21"],
        ["2024-08-09", "2024-09-02"],
      ],
      "polarization": "vv"
    },
  ),
  connection=connection,
).save_result("GTiff")

job = datacube.create_job(title="sample_preprocessing_vv")
job.start_job()

<BatchJob job_id='j-250605090458430bb8a21919d97798a0'>

In [2]:
job = connection.job("j-2505291159174cb4ae73808db86f96ff")
job

<BatchJob job_id='j-2505291159174cb4ae73808db86f96ff'>

Re use the pre-processed Sentinel-1 SLC data via STAC:

In [3]:
job = connection.job("j-2505291159174cb4ae73808db86f96ff")
stac_result = job.get_results().get_metadata()
stac_result_url = [x["href"] for x in stac_result["links"] if x["rel"]=="canonical"][0]
stac_result_url

'https://openeo.dataspace.copernicus.eu/openeo/1.2/jobs/j-2505291159174cb4ae73808db86f96ff/results/NWFkOThhZjAtZDQ2ZS00MDk0LTg0Y2EtZGJjM2FlMjllNmE5/0f7319bebbdd9ecf308bfc6183abee55?expires=1749718768'

In [4]:
cube = connection.load_stac(stac_result_url)

In [5]:
from openeo.processes import ProcessBuilder, array_element

def compute_real_part_numerator_VV(x:ProcessBuilder):
    i_mst = array_element(x,label="i_VV_mst")
    q_mst = array_element(x,label="q_VV_mst")
    i_slv = array_element(x,label="i_VV_slv")
    q_slv = array_element(x,label="q_VV_slv")
    return (i_mst*i_slv + q_mst*q_slv)

def compute_imag_part_numerator_VV(x:ProcessBuilder):
    i_mst = array_element(x,label="i_VV_mst")
    q_mst = array_element(x,label="q_VV_mst")
    i_slv = array_element(x,label="i_VV_slv")
    q_slv = array_element(x,label="q_VV_slv")
    return (q_mst*i_slv - i_mst*q_slv)

def compute_real_part_numerator_VH(x:ProcessBuilder):
    i_mst = array_element(x,label="i_VH_mst")
    q_mst = array_element(x,label="q_VH_mst")
    i_slv = array_element(x,label="i_VH_slv")
    q_slv = array_element(x,label="q_VH_slv")
    return (i_mst*i_slv + q_mst*q_slv)

def compute_imag_part_numerator_VH(x:ProcessBuilder):
    i_mst = array_element(x,label="i_VH_mst")
    q_mst = array_element(x,label="q_VH_mst")
    i_slv = array_element(x,label="i_VH_slv")
    q_slv = array_element(x,label="q_VH_slv")
    return (q_mst*i_slv - i_mst*q_slv)

def compute_mst_denominator_VV(x:ProcessBuilder):
    i_mst = array_element(x,label="i_VV_mst")
    q_mst = array_element(x,label="q_VV_mst")
    return (i_mst**2+q_mst**2)

def compute_slv_denominator_VV(x:ProcessBuilder):
    i_slv = array_element(x,label="i_VV_slv")
    q_slv = array_element(x,label="q_VV_slv")
    return (i_slv**2+q_slv**2)

def compute_mst_denominator_VH(x:ProcessBuilder):
    i_mst = array_element(x,label="i_VH_mst")
    q_mst = array_element(x,label="q_VH_mst")
    return (i_mst**2+q_mst**2)

def compute_slv_denominator_VH(x:ProcessBuilder):
    i_slv = array_element(x,label="i_VH_slv")
    q_slv = array_element(x,label="q_VH_slv")
    return (i_slv**2+q_slv**2)

def compute_coherence_amplitude_VV(x:ProcessBuilder):
    q_num_VV = array_element(x,label="q_num_VV")
    i_num_VV = array_element(x,label="i_num_VV")
    mst_den = array_element(x,label="mst_den")
    slv_den = array_element(x,label="slv_den")
    
    coh_i = i_num_VV / ((mst_den * slv_den + 1e-12)**0.5)
    coh_q = q_num_VV / ((mst_den * slv_den + 1e-12)**0.5)
    
    return (coh_i**2+coh_q**2)**0.5

def compute_coherence_amplitude_VH(x:ProcessBuilder):
    q_num_VH = array_element(x,label="q_num_VH")
    i_num_VH = array_element(x,label="i_num_VH")
    mst_den = array_element(x,label="mst_den")
    slv_den = array_element(x,label="slv_den")
    
    coh_i = i_num_VH / ((mst_den * slv_den + 1e-12)**0.5)
    coh_q = q_num_VH / ((mst_den * slv_den + 1e-12)**0.5)
    
    return (coh_i**2+coh_q**2)**0.5

In [11]:
dates = [np.datetime64("2024-08-09"),np.datetime64("2024-08-21"),np.datetime64("2024-09-02")]
dates_pair = [[dates[0],dates[1]],[dates[1],dates[2]]]
# We use a loop over the dates we want to request and a combination of filter_temporal + merge_cubes to get only the dates we want:
for i,date_pair in enumerate(dates_pair):
    # The requested date until the day after, since openEO excludes the end date
    mst_date = date_pair[0]
    slv_date = date_pair[1]
    mst_temporal_extent = [str(mst_date),str(mst_date + np.timedelta64(1, "D"))]
    slv_temporal_extent = [str(slv_date),str(slv_date + np.timedelta64(1, "D"))]

    if i == 0:
        S1_slant_range_mst_bands = cube.filter_temporal(mst_temporal_extent)
        S1_slant_range_slv_bands = cube.filter_temporal(slv_temporal_extent)
        break # +++++++++++++++++++++ Do it only for one pair for testing
    else:
        S1_slant_range_mst_bands = cube.filter_temporal(mst_temporal_extent).merge_cubes(S1_slant_range_mst_bands)
        S1_slant_range_slv_bands = cube.filter_temporal(slv_temporal_extent).merge_cubes(S1_slant_range_slv_bands)
        

In [12]:
S1_slant_range_mst_bands = S1_slant_range_mst_bands.filter_bands([0,1]).drop_dimension("t") # "i_" real part,"q_" imaginary part
S1_slant_range_slv_bands = S1_slant_range_slv_bands.filter_bands([0,1]).drop_dimension("t")

In [13]:
S1_slant_range_mst_bands = S1_slant_range_mst_bands.rename_labels(dimension="bands",
                                                                  source=[0,1],
                                                                  target=["i_VV_mst","q_VV_mst"])
S1_slant_range_slv_bands = S1_slant_range_slv_bands.rename_labels(dimension="bands",
                                                                  source=[0,1],
                                                                  target=["i_VV_slv","q_VV_slv"])

In [14]:
S1_slant_range_mst_slv_bands = S1_slant_range_mst_bands.merge_cubes(S1_slant_range_slv_bands)

In [15]:
i_numerator_VV = S1_slant_range_mst_slv_bands.reduce_dimension(dimension="bands",
                                                               reducer=compute_real_part_numerator_VV)
q_numerator_VV = S1_slant_range_mst_slv_bands.reduce_dimension(dimension="bands",
                                                               reducer=compute_imag_part_numerator_VV)

In [16]:
mst_denominator_VV = S1_slant_range_mst_slv_bands.reduce_dimension(dimension="bands",
                                                                   reducer=compute_mst_denominator_VV)
slv_denominator_VV = S1_slant_range_mst_slv_bands.reduce_dimension(dimension="bands",
                                                                   reducer=compute_slv_denominator_VV)

In [17]:
i_numerator_VV_with_dim = i_numerator_VV.add_dimension(type="bands",name="bands",label="i_num_VV")
q_numerator_VV_with_dim = q_numerator_VV.add_dimension(type="bands",name="bands",label="q_num_VV")
mst_denominator_VV_with_dim = mst_denominator_VV.add_dimension(type="bands",name="bands",label="mst_den")
slv_denominator_VV_with_dim = slv_denominator_VV.add_dimension(type="bands",name="bands",label="slv_den")

coh_numerator_denominator_VV = i_numerator_VV_with_dim.merge_cubes(q_numerator_VV_with_dim).merge_cubes(mst_denominator_VV_with_dim).merge_cubes(slv_denominator_VV_with_dim)

In [ ]:
# coh_numerator_denominator_VV.download("coh_numerator_denominator_VV.nc")

In [ ]:
# Boxcar filter
range_dim   = 5
azimuth_dim = 1

# Create kernel
kernel = np.ones((range_dim,azimuth_dim))

# Pad with zeros to get a square kernel
pad = int((range_dim - azimuth_dim) / 2)
kernel = np.pad(kernel,((0,0),(pad,pad)))

coh_numerator_denominator_VV_boxcar = coh_numerator_denominator_VV.apply_kernel(kernel=kernel,
                                                                                factor=1/(range_dim*azimuth_dim),
                                                                                border=0)

coh_VV = coh_numerator_denominator_VV_boxcar.reduce_dimension(dimension="bands",
                                                              reducer=compute_coherence_amplitude_VV)
coh_amplitude = coh_VV.add_dimension(name="bands",label="coh_VV_amp")
coh_amplitude = coh_amplitude.save_result(format="GTiff")

job = coh_amplitude.create_job(title="ClouDInSAR_coherence_from_preprocessing")

In [21]:
job.start_job()

<BatchJob job_id='j-25060509012543ebb48aa5ee8c65b7ed'>

In [ ]:
job = connection.job("j-25060509012543ebb48aa5ee8c65b7ed")
job.get_results().download_files()